In [133]:
import json
from pprint import pprint
from termcolor import colored
from colored import fg, bg, attr

data_file = "../predict_88_acc.jsonl"
id2name = "../data/id2name.json"

NUMBER_OF_SAMPELS = 100

color = bg('yellow_1') + fg('black')
reset = attr('reset')
blue = "blue"
green = "green"

print()

with open(id2name, 'r') as fp:
    id2name = json.load(fp)

def in_pos(indx, positions):
    for ent_pos in positions:
        if ent_pos[0] <= indx < ent_pos[1]:
            return True

    return False

In [134]:
def change_word_by_entity(entity_str, this_sentence,i,color):
        if list(map(lambda x: x.lower(), this_sentence[i:i + len(entity_str.split(" "))])) == entity_str.split():
            this_sentence = this_sentence[:i] + [colored(w, color, attrs=['bold']) for w in entity_str.split()] + \
                            this_sentence[i + len(entity_str.split(" ")):]
        return this_sentence


In [135]:
preffix = ""
def _show_sentence(head,tail,this_sentence,relation,correct_relation = None, is_prediction = False):
#     print(preffix + "head:", colored(head, 'blue', attrs=['bold']), "  ,  ", "tail", colored(tail, 'green', attrs=['bold']))          
    this_sentence = this_sentence.split()
    for i, word in enumerate(this_sentence):
            if word.lower() == head.split()[0]:
                this_sentence = change_word_by_entity(head, this_sentence,i,blue)
            if word.lower() == tail.split()[0]:
                this_sentence = change_word_by_entity(tail, this_sentence,i,green)
    color = 'yellow' if correct_relation is None or relation == correct_relation else 'grey'
    if is_prediction: color = "red"
    print(preffix + colored(relation, color,attrs=['bold']) + "\t", " ".join(this_sentence))

In [136]:
def show_set(data):
    correct_relation = data['correct_relation']
    prediction = data["prediction"]
    query = data['query']
    head = data['query_head']
    tail = data['query_tail']
    _show_sentence(head,tail,query,correct_relation)
    print()

    for k in range(5):
        this_sentence = data["sentence_" +str(k)]
        head = data['sentence_'+str(k)+'_head']
        tail = data['sentence_'+str(k)+'_tail']
        relation = data['sentence_'+str(k)+'_relation']
        _show_sentence(head,tail,this_sentence,relation,correct_relation,  k == int(prediction))
    print()
    return correct_relation
        

In [137]:
total_wrong = 0
total_correct = 0
result_per_type = {}
with open(data_file) as f:
    for ex_index,line in enumerate(f,1):
        data = json.loads(line)
        the_gold_rel = data['correct_relation']
        if data['answer'] == "wrong":
            total_wrong +=1
            print("wrong example number: ",total_wrong,  "example number: ", ex_index)
            show_set(data)
            result_per_type[the_gold_rel] = result_per_type.get(the_gold_rel,[[],[]])
            result_per_type[the_gold_rel][0].append(data)
            
            print()
        else:
            result_per_type[the_gold_rel] = result_per_type.get(the_gold_rel,[[],[]])
            result_per_type[the_gold_rel][1].append(data)
            total_correct +=1





wrong example number:  1 example number:  1
member of	 Being member of european economic area ( EEA ) , Norway implemented the european union directive 2002/24/EC .

sport	 benoît laffineur ( born 30 October 1957 ) is a French former swimmer .
follows	 Wilson was re - elected in the 2003 , 2006 , and 2009 general elections .
member of	 The inaugural Lord Lewis Prize was awarded to lord may of oxford , former President of the royal society and Chief Scientific Advisor to the UK Government , in 2008 .
mother	 In 579 , her stepson Hermenegild married her granddaughter Ingund , daughter of goiswintha 's daughter brunhilda by the Frankish king Sigebert I.
original language of film or TV show	 The film stars the familiar characters from the finnish TV show " the autocrats " in a fairy tale where the inhabitants of a small finnish village have to defend themselves against a despotic emperor .


wrong example number:  2 example number:  2
crosses	 The route crosses the minnesota river at the c

member of	 Penn finished the season 7–3 , 6–1 in ivy league play to finish in a tie with princeton for the ivy league title .


wrong example number:  128 example number:  995
member of	 With fraternity brothers William Packer and Gregory Anderson producing , rob hardy wrote and directed " Chocolate City " , a $ 20,000 film that Blockbuster distributed through its video stores .

original language of film or TV show	 pedda gedda ( kannada : ಪೆದ್ದ ಗೆದ್ದ ) is a 1982 Indian kannada film , directed by H. R. Bhargava and produced by Dwarakish .
mother	 When marianus v , the youngest son of Beatrice ' elder sister eleanor , died in 1407 , Arborea experienced a succession crisis .
sport	 Bebb was born in Church Village , Mid Glamorgan , and is the son of former Welsh international rugby player dewi bebb .
member of	 He was involved in the society of antiquaries 's report on the restorations and alterations at Bath by charles davis in the 1880s .
located in or next to body of water	 The Dutch 

wrong example number:  265 example number:  2147
main subject	 He was a regular stuntman on the world war ii TV series " the rat patrol " and a double for action actors Sean Connery and William Smith .

located in or next to body of water	 Following the Italian Armistice in early September 1943 , " Albona " and " Rovigno " were captured by the Germans at the island of syros in the aegean sea on 10 September .
main subject	 The importance of neutron capture to the observed abundance of the chemical elements was first described in 1957 in the b2fh paper .
constellation	 ε Monocerotis , Latinised as epsilon monocerotis , is the Bayer designation of a binary star system in the equatorial constellation monoceros .
part of	 Above the south porch are the symbols of the four evangelists ( Saints Matthew , mark , Luke and John ) .
crosses	 In March 2007 , construction commenced on what would be the longest road bridge to span the zambezi .


wrong example number:  266 example number:  2156
part

In [138]:
print(total_wrong/(total_correct+total_wrong))

0.1212


In [142]:
ex_index

2500

In [140]:
for k,v in result_per_type.items():
    print("acc for ", k)
    print(len(v[1])/ (len(v[0])+len(v[1]) ))

acc for  member of
0.5714285714285714
acc for  crosses
0.9515151515151515
acc for  part of
0.3897058823529412
acc for  mother
0.967741935483871
acc for  child
0.9342105263157895
acc for  main subject
0.7152777777777778
acc for  military rank
0.993103448275862
acc for  located in or next to body of water
0.8580645161290322
acc for  voice type
1.0
acc for  follows
0.9518072289156626
acc for  original language of film or TV show
1.0
acc for  position played on team / speciality
1.0
acc for  spouse
0.75
acc for  competition class
0.9939024390243902
acc for  constellation
1.0
acc for  sport
0.8930817610062893


In [95]:
for i, sample in enumerate(data):
    bool_print_title = True

    if i == NUMBER_OF_SAMPELS:
        break

    for lab in sample['labels']:
        evidence_list = lab['evidence']

        if len(evidence_list) > 1:

            if bool_print_title:
                print("########################################################################################################")
                print()
                print(color + sample['title'] + reset)
                print()
                bool_print_title = False


            head = sample['vertexSet'][lab['h']]
            tail = sample['vertexSet'][lab['t']]

            head_names = [ent['name'] for ent in head]
            tail_names = [ent['name'] for ent in tail]

            print("head:", colored(head_names[0], 'blue', attrs=['bold']), "  ,  ", "tail:", colored(tail_names[0], 'green', attrs=['bold']))
            print()
            print("rel:", colored(id2name[lab['r']], 'yellow', attrs=['bold']))
            print()


            for k, sent_indx in enumerate(evidence_list):
                head_pos = [ent['pos'] for ent in head if ent["sent_id"] == sent_indx]
                tail_pos = [ent['pos'] for ent in tail if ent["sent_id"] == sent_indx]

                sent = ""
                for j, word in enumerate(sample['sents'][sent_indx]):
                    if in_pos(j, head_pos):
                        word = colored(word, 'blue', attrs=['bold'])

                    if in_pos(j, tail_pos):
                        word = colored(word, 'green', attrs=['bold'])

                    sent += word + " "


                for name in head_names:
                    sent = sent.replace(name, colored(name, 'blue', attrs=['bold']))

                for name in tail_names:
                    sent = sent.replace(name, colored(name, 'green', attrs=['bold']))


                print(colored("sent: " + str(k), 'red'), sent)

                # print(sample['sents'][sent_indx])

            print()
            print("***************************************************************************************************")
            print()



TypeError: string indices must be integers